# Upper tropospheric humidity: Climatology of global the upper tropospheric humidity and uncertainty

## Import packages

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

## Define Parameters

In [ ]:
# Variables
variables = ["uth_mean_ascend_descend"]
assert set(variables) <= {
    "n_obs_all_ascend",
    "n_obs_all_ascend_descend",
    "n_obs_all_descend",
    "n_obs_valid_ascend",
    "n_obs_valid_ascend_descend",
    "n_obs_valid_descend",
    "tb18_mean_ascend",
    "tb18_mean_descend",
    "tb18_std_ascend",
    "tb18_std_descend",
    "uth_mean_ascend",
    "uth_mean_ascend_descend",
    "uth_mean_descend",
    "uth_median_ascend",
    "uth_median_ascend_descend",
    "uth_median_descend",
    "uth_std_ascend",
    "uth_std_ascend_descend",
    "uth_std_descend",
}

# Sensors
sensors = [
    "amsu_b_on_noaa_15",
    "amsu_b_on_noaa_16",
    "amsu_b_on_noaa_17",
    "mhs_on_metop_a",
    "mhs_on_metop_b",
    "mhs_on_metop_c",
    "mhs_on_noaa_18",
    "mhs_on_noaa_19",
]
assert set(sensors) <= {
    "amsu_b_on_noaa_15",
    "amsu_b_on_noaa_16",
    "amsu_b_on_noaa_17",
    "mhs_on_metop_a",
    "mhs_on_metop_b",
    "mhs_on_metop_c",
    "mhs_on_noaa_18",
    "mhs_on_noaa_19",
}

## Define Request

In [ ]:
collection_id = "satellite-upper-troposphere-humidity"
request = {
    "variable": "all",
    "format": "zip",
}
time_dict = {
    "amsu_b_on_noaa_15": {"start": "1999-01", "end": "2002-12"},
    "amsu_b_on_noaa_16": {"start": "2001-01", "end": "2006-12"},
    "amsu_b_on_noaa_17": {"start": "2003-01", "end": "2008-12"},
    "mhs_on_metop_a": {"start": "2007-01", "end": "2021-02", "skip": ["2014-04"]},
    "mhs_on_metop_b": {"start": "2013-01", "end": "2021-02"},
    "mhs_on_metop_c": {"start": "2019-12", "end": "2021-02"},
    "mhs_on_noaa_18": {"start": "2006-01", "end": "2018-10"},
    "mhs_on_noaa_19": {"start": "2016-01", "end": "2021-02"},
}

## Download and transform

In [ ]:
kwargs = {
    "chunks": {"year": 1, "month": 1},
    "transform_chunks": False,
    "transform_func_kwargs": {"weights": False},
    # Parameters to speed up I/O
    "concat_dim": "time",
    "combine": "nested",
    "data_vars": "minimal",
    "coords": "minimal",
    "compat": "override",
    "parallel": True,
}

frequencies = ("time", "seasonal")
datasets_dict = {freq: [] for freq in frequencies}
for sensor in sensors:
    months = pd.date_range(
        time_dict[sensor]["start"], time_dict[sensor]["end"], freq="1MS"
    ).strftime("%Y-%m")
    requests = []
    for month in months:
        if month not in time_dict[sensor].get("skip", []):
            requests.extend(
                download.update_request_date(
                    request | {"sensor_on_satellite": sensor},
                    month,
                    month,
                    stringify_dates=True,
                )
            )
    sensor = sensor.upper().replace("_ON_", " on ").replace("_", "-")
    for freq in frequencies:
        for reduction in ("mean", "std"):
            print(f"{sensor=} {freq=} {reduction=}")
            transform_func = getattr(diagnostics, f"{freq}_weighted_{reduction}")
            ds = download.download_and_transform(
                collection_id, requests, transform_func=transform_func, **kwargs
            )
            datasets_dict[freq].append(
                ds.expand_dims(
                    sensor=[sensor],
                    reduction=[reduction],
                )
            )
datasets_dict = {k: xr.merge(v) for k, v in datasets_dict.items()}

## Plot maps

In [ ]:
for freq, ds in datasets_dict.items():
    for variable in variables:
        da = ds[variable]
        for sensor, da_sensor in da.groupby("sensor"):
            for reduction, da_reduction in da_sensor.groupby("reduction"):
                plot.projected_map(
                    da_reduction,
                    projection=ccrs.Robinson(),
                    cmap="YlGnBu" if reduction == "mean" else "Purples",
                    robust=True,
                    center=False,
                    levels=9,
                    extend="both",
                    col="season" if freq == "seasonal" else None,
                    col_wrap=2,
                )
                title = f"{reduction=} {sensor=}"
                plt.suptitle(title) if freq == "seasonal" else plt.title(title)
                plt.show()